<table>
  <tr>
    <td><img src="https://github.com/rvss-australia/RVSS/blob/main/Pics/RVSS-logo-col.med.jpg?raw=1" width="400"></td>
    <td><div align="left"><font size="30">Finding blobs</font></div></td>
  </tr>
</table>

(c) Peter Corke 2024

Robotics, Vision & Control: Python, see section 12.1.3.4

## Configuring the Jupyter environment
We need to import some packages to help us with linear algebra (`numpy`), graphics (`matplotlib`), and machine vision (`machinevisiontoolbox`).
If you're running locally you need to have these packages installed.  If you're running on CoLab we have to first install machinevisiontoolbox which is not preinstalled, this will be a bit slow.

In [ ]:
try:
    import google.colab
    print("Running on CoLab")
    !pip install machinevision-toolbox-python
    COLAB = True
except:
    COLAB = False

%matplotlib ipympl
import matplotlib.pyplot as plt
import matplotlib
matplotlib.rcParams["figure.figsize"] = [8, 8]

import numpy as np
np.set_printoptions(linewidth=120, formatter={"float": lambda x: f"{x:8.4g}" if abs(x) > 1e-10 else f"{0:8.4g}"})

# display result of assignments
%config ZMQInteractiveShell.ast_node_interactivity = "last_expr_or_assign"

from machinevisiontoolbox import Image
from spatialmath import SE3
from spatialmath import base
import dot

***

# Find objects in a binary image

We start by loading a binary image

In [ ]:
sharks = Image.Read("./shark2.png")
sharks.disp();

When we look at this we see two white objects, vaguely-shark shaped, against a black background.  But the *objects* are a mental construct of ours, the image is simply 250,000 black and white pixels.  How do group adjacent pixels of the same color to find the objects?

This is a very classical computer vision problem that goes by many names: blob analysis, connectivity analysis, region labelling and many more.
Such objects in a binary object are often called *blobs*.

Using this toolbox we simply write

In [ ]:
blobs = sharks.blobs()

The the result is a feature object that describes the *blobs* present in the scene.
In this case there are


In [ ]:
len(blobs)

Each blob has a number of properties which are shown in the columns of the table.  Each blob has a unique `id` and a `parent` which will discuss later.

We can put a box around the blobs and label them with their `id` number.

In [ ]:
sharks.disp(block=None)
blobs.plot_labelbox(color="yellow", linewidth=2)

The background is also a blob, a black blob, but here we are ignoring that.

This `blobs` object can be indexed or sliced just like a list.  Each element has a number of properties which were listed in the table earlier.  For example, its centroid (centre of mass) is

In [ ]:
blobs[0].centroid

its area in pixels

In [ ]:
blobs[0].area

and a bounding box

In [ ]:
blobs[0].bbox

where the first row is the u-axis range, and the second row is the v-axis range.  Alternatively we can consider the columns: the first column is the top-left coordinate and the second column is the bottom-right coordinate.

For each blob we can obtain the length of its perimeter

In [ ]:
blobs[0].perimeter_length

as well as the perimeter, as a set of points

In [ ]:
blobs[0].perimeter

A simple but useful measure of "shape" is circularity, computed from area and perimeter.  It is 1 for a circle and 0 for a line.

In [ ]:
blobs[0].circularity

These properties can also be computed on a list of blob objects, and the result is an array or list. For example

In [ ]:
blobs.area

In [ ]:
blobs.centroid

The blob objects also support some graphical operations.
which depicts and labels each blob.  We also marked the centroids.

In [ ]:
sharks.disp(block=None)
blobs.plot_box(color="yellow", linewidth=2)
blobs.plot_centroid()

Now we will load a more complex image that has blobs with holes that contain blobs with holes...

In [ ]:
multi = Image.Read("multiblobs.png", grey=True)
multi.disp();

**Q: how many blobs are in this scene?**

We can find the blobs

In [ ]:
blobs = multi.blobs()
len(blobs)

and display the parameters as a table

In [ ]:
print(blobs)

We note that this time some of the blobs have a parent that is not -1 (the background blob).  Looking at the figure we can see that there is some hierarchy of blobs -- a blob can have subblobs, which themselves can have subblobs.  We often refer to subblobs as **child** blobs, and they are contained within a **parent** blob.

The parent of blob 2 is

In [ ]:
blobs[2].parent

And the children of blob 2 are

In [ ]:
blobs[1].children

The datastructure we need to represent this hierarchy is a tree.  If you have the `graphviz` package installed and the `dot` command-line utility then we can run

In [ ]:
blobs.dotfile(show=True);

The final thing we will do is to create a label image and display it

In [ ]:
labels = blobs.label_image()
labels.disp(
    colormap="viridis",
    ncolors=10,
    colorbar=dict(shrink=0.8, aspect=20 * 0.8),
    block=True,
)

Where the value of each pixel is the `id` of the blob it belongs to.  By drifting the cursor over the image you can see which pixels belong to say blob #6.  The image uses a colorful colormap to make it easy to see the different label areas.